In [ ]:
%load_ext autoreload
%autoreload 2
import sys 
import numpy as np

# RNN (Recurrent Neural Networks)
Articulos recomendado: 
- http://karpathy.github.io/2015/05/21/rnn-effectiveness/
- http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- http://blog.echen.me/2017/05/30/exploring-lstms/

## Que tienen de nuevo respecto a MLP y CNN?
- MLP y CNN solo aceptan un vector de entrada de tamaño fijo y devuelve un vector de salida de tamaño fijo
- RNN trabjan con secuencias tanto a la entrada como a la salida
- No tienen por que estrictamente tener una secuencia ni a la entrada ni a la salidad. De hecho hasta podrían no tener nada a la "entrada"

<src img="rnn_types.jpeg">

Aclaraciones: 
- Cada cuadrado NO es una neurona si no una capa que puede contener N neuronas
- Cada flecha representa la interconexión entre dos capas. Los pesos forman una matriz de la N1xN2 donde N1 y N2 son la cantidad de neuronas en cada capa respectivamente.

## Tipos

<img src="imgs/rnn_types.jpeg">

- **One to One**: CNN, MLP
- **One to many**: [Image captioning](https://www.youtube.com/watch?v=xKt21ucdBY0)
- **Many to one**: Sentiment Analisys, Detectar voz de hombre vs voz de mujer
- **Many to Many** [(sequence to sequence)](https://youtu.be/dkHdEAJnV_w): [Traducción](https://github.com/jganzabal/aind2-nlp-capstone/blob/master/machine_translation.ipynb) Dimensión de entrada differente a la de salida.
- **Many to Many Sincronizado**: Etiquetado de tramas de video, POS (Part of Speech) cada palabra se clasifica en verbo, articulo, etc, speech2text, text2speech, NER (Named Entity Recognition). Dimensión de entrada igual a dimensión de salida

# Modelos de lenguaje Generativos:
- Predecir la proxima palabra en funcion de las anteriores
- Predecir el proximo caracter en función de los anteriores

**Resultado**: Probabilidad dada la secuencia de caracteres o de palabras

**Aplicaciones de los modelos de lenguaje** (Mas allá de la posibilidad de generar texto):
- OCR
- Speach2Text
- Detección de autores

## "If training vanilla neural nets is optimization over functions, training recurrent nets is optimization over programs."
- Es una secuencia de ejecución mas que una clasificación
- Las RNN son Turing completo en principio [Turing Complete](https://en.wikipedia.org/wiki/Turing_completeness), [RNN Turing complete](http://binds.cs.umass.edu/papers/1995_Siegelmann_Science.pdf)

# Detalles de la arquitectura:

## Unidad de Elman o RNN unit

**Nota importante**: Cada unidad no es una neurona sino que una capa

<img src="imgs/RNN_vs_FNN.png">

¿Cual es el tamaño de $W_h$?

Todo se conecta con todo -> Si hay M hidden units tenemos $M^2$ $W_h$s 

## En ecuaciones:

### FFN

$h_t = f(W_x^T X_t + b_h)$

$y_t = softmax(W_o^T h_t + b_o)$

$f$ usualmente RELU, sigmoidea, etc

### RNN

$h_t = f(W_h^T h_{t-1} + W_x^T X_t + b_h)$

$y_t = softmax(W_o^T h_t + b_o)$

$f$ es tanh usualmente pero puede ser RELU, sigmoid, etc

## Con keras

In [5]:
from keras.layers import SimpleRNN, Dense
from keras.models import Sequential

Using TensorFlow backend.


In [6]:
rnn_neurons = 2
time_steps = 1000 # T
n_features = 3 # D
input_shape = (time_steps, n_features)
model = Sequential()
model.add(SimpleRNN(rnn_neurons, input_shape = input_shape))
model.add(Dense(3))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 2)                 12        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 9         
Total params: 21
Trainable params: 21
Non-trainable params: 0
_________________________________________________________________


### Que es time_steps?

## Desplegando (Unfolding) la RNN
#### BPTT (Back-Propagation Through Time)
- Secuencia de longitud 5

<img src="imgs/unfold_RNN.png">

Observaciones:
- Es como una FNN con 5 hidden layers pero con pesos compartidos (Shared weights): $W_h, W_o, W_x$
- Donde habiamos visto pesos compartidos?
- Se puede pensar como si h fuera la entrada y X es una señal de control en cada paso

Preguntas:
- Son todas las Y importantes? En que casos?
- Que diferencia hay entre estado interno (internal state) de la RNN y los pesos?
- Cada cuanto se resetea el estado interno? Y los pesos?

# Detalles de la entrada $X$

## Entrada en Vanilla Networks:

NxD, donde N es la cantidad de muestras y D es la cantidad de features

In [7]:
import numpy as np

In [8]:
# Ejemplo N = 3, D = 2
Ex_1 = np.array([[0.5, 0.3], [0.2, 0.1], [0.7, 0.3]])
print(Ex_1)
print(Ex_1.shape)

[[0.5 0.3]
 [0.2 0.1]
 [0.7 0.3]]
(3, 2)


## Entrada en RNNs:

Secuencias de longitud fija

NxTxD, donde T es la longitud de la secuencia

In [9]:
# Ejemplo N = 4, T = 3, D=2
Ex_2 = np.array([[[0.5, 0.3], [0.2, 0.1], [0.7, 0.3]], [[0.54, 0.1], [0.23, 0.3], [0.9, 0.1]], [[0.5, 0.3], [0.2, 0.1], [0.7, 0.3]], [[0.54, 0.1], [0.23, 0.3], [0.9, 0.1]]])
print(Ex_2)
print(Ex_2.shape)

[[[0.5  0.3 ]
  [0.2  0.1 ]
  [0.7  0.3 ]]

 [[0.54 0.1 ]
  [0.23 0.3 ]
  [0.9  0.1 ]]

 [[0.5  0.3 ]
  [0.2  0.1 ]
  [0.7  0.3 ]]

 [[0.54 0.1 ]
  [0.23 0.3 ]
  [0.9  0.1 ]]]
(4, 3, 2)


Secuencias de longitud variable

In [10]:
Ex_3 = np.array([[[0.5, 0.3], [0.7, 0.3]], [[0.54, 0.1], [0.23, 0.3], [0.9, 0.1]], [[0.7, 0.3]], [[0.54, 0.1], [0.23, 0.3], [0.9, 0.1]]])
print(Ex_3)
print(Ex_3.shape)
print(np.array(Ex_3[0]).shape)
print(np.array(Ex_3[1]).shape)
print(np.array(Ex_3[2]).shape)

[list([[0.5, 0.3], [0.7, 0.3]])
 list([[0.54, 0.1], [0.23, 0.3], [0.9, 0.1]]) list([[0.7, 0.3]])
 list([[0.54, 0.1], [0.23, 0.3], [0.9, 0.1]])]
(4,)
(2, 2)
(3, 2)
(1, 2)


En tensorflow es posible, en keras se suele hacer padding

¿Que pasa cuando tenemos longitudes distintas? Analizar el unfolding

### Desventajas de padding
- Podría haber una secuencia de longitud mayor en el test set. Que hacemos con eso?
- Es probable que los casos de secuencia largas sean poco probables por lo que realizaremos multiplicaciones de matrices innecesarias

# Resumiendo:
## RNN vs Vanilla Network
- Menor cantidad de parametros para resolver lo mismo
- En principio una FFN podria resolver con la misma precisión un problema resuelto por una RNN, pero es demasiado complejo encontrar la solución
- La estructura de la RNN simplifica la tarea de encontrar una solución de manera eficiente
- Ya sabemos que el problema tiene naturaleza recurrente (necesitamos memoria)
- FNN tienen que tener tamaño fijo en contraste con las RNNs

## Las RNN pueden resolver problemas muy diferentes desde el punto de vista del tipo de procesamiento sobre la secuencia
- Secuencia temporal modelada con "longitud infinita": Valor de acciones - Modelos tipo AR(N)
- Secuencias finitas "independientes" unas de otras: Sentiment analisys (Longitud variable), paridad (Longitud fija)
- Modelos de lenguaje: Por caracter o por palabra
- Seq2Seq: Traducción, chatbots, Image captioning

[](BiLSTM-Image-classif)